## Scikit-learn 라이브러리
* 사이킷 런 : 대표적인 머신러닝 라이브러리
* 분류, 회귀, 군집(클러스터링) 등 다양한 알고리즘 제공
* 아나콘다에 포함  
    * train_test_split(data_df, test_size)
    * KNeighborsClassifier(n_neighbors)
    * svm.SVC(C, gamma)
    * metrics.accuracy_score(test_label, predict_result)

## NBA 농구선수 게임 기록 데이터로 포지션 머신러닝(SVM, KNN)으로 예측
* Pos : 포지션
* 3P : 한 경기 평균 3점 슛 성공 횟수
* TRB : 한 경기 리바운드 성공 횟수
* BLK : 한 경기 블로킹 성공 횟수

### 1. 데이터 획득
basketball_state.csv 파일의 데이터를 DataFrame으로 불러오기
* **pd.read_csv()**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split # 8:2로 학습:테스트 쪼개줌
from sklearn.neighbors import KNeighborsClassifier #KNN 모델 
from sklearn import svm   #SVM 모델
from sklearn import metrics  #정확도 비교

In [2]:
df = pd.read_csv('./basketball_stat.csv') # DF
data_df = df[0:200] # 전체 데이터셋 중 200개 모두 사용 / # DF
# 행 이름이 아니라 슬라이싱으로 가져올때는 .iloc 안해도 됨! 그냥 슬라이싱처럼!

print(type(df), type(data_df))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


### 2. 전체 데이터셋 나누기 (split)
train set(학습 데이터)과 test set(테스트 데이터)로 나누기 => 8:2
* from sklearn.model_selection import train_test_split
* **train_test_split(df, test_size = num)** 

In [3]:
train, test = train_test_split(data_df, test_size= 0.2) # train:test = 8:2
print("train data", train.shape)
print("test data",test.shape)

train data (160, 8)
test data (40, 8)


### 3. train 데이터셋 분리
학습 데이터와 레이블(정답) 분리해주기
* train_data_df = train[['3P', '2P', 'BLK']] : **input**
    * train_data_df.**values**  

  
* train_label_df = train[['pos']] : **output**
    * train_label_df.**values.ravel()**

In [4]:
train_data_df = train[['3P','2P', 'BLK']] # 학습시 3개의 열만 가지고 학습
train_label_df = train[['Pos']]   # 정답 label 열 설정 (pos을 예측하는 것!)

# 학습할 때에는 DataFrame형태가 아닌, **값들만** 가지고 학습하는 것!!
train_data = train_data_df.values  # 값부분만 가져오기(2차원 형태)
train_label = train_label_df.values.ravel() # 값부분을 1차원으로 펴주기

### 4. test 데이터셋 분리
학습데이터와 레이블(정답) 분리해주기
* test_data_df = test[['3P', '2P', 'BLK']] : **input**
    * test_data_df.**values**  

  
* test_label_df = test[['pos']] : **output**
    * test_label_df.**values.ravel()**

In [5]:
test_data_df = test[['3P','2P', 'BLK']] 
test_label_df = test[['Pos']]  

# 학습할 때에는 DataFrame형태가 아닌, 값들만 가지고 학습하는 것!!
test_data = test_data_df.values # 값부분만 가져오기(2차원 형태)
test_label = test_label_df.values.ravel() #값부분을 **1차원으로 펴주기**

In [6]:
test_data_df.head() # DataFrame 형태

,3P,2P,BLK
128,1.5,4.0,0.4
15,0.4,1.6,0.7
132,1.0,3.2,0.9
29,1.4,2.6,0.3
63,0.3,2.2,0.5


In [7]:
test_data # 값부분만(values) 가져온 것 (df말고 일반 2차원 배열으로)

array([[1.5, 4. , 0.4],
       [0.4, 1.6, 0.7],
       [1. , 3.2, 0.9],
       [1.4, 2.6, 0.3],
       [0.3, 2.2, 0.5],
       [0.1, 4.1, 1.5],
       [2.7, 2.8, 0.4],
       [0.5, 2.1, 0.1],
       [0.2, 4.6, 1.4],
       [0.9, 5.1, 1.5],
       [0.2, 4. , 0.6],
       [1.4, 1.4, 0.6],
       [0. , 6.6, 1.3],
       [0. , 5.9, 2.3],
       [0.3, 0.8, 0.3],
       [2.1, 4.9, 0.3],
       [0.2, 1.2, 0.5],
       [1. , 1.6, 0.1],
       [0. , 1.1, 0.4],
       [0. , 3.1, 2.4],
       [2.5, 6.8, 0.7],
       [0.3, 1. , 0. ],
       [0. , 3.2, 0.9],
       [0.1, 0.3, 0.1],
       [0.3, 0.6, 0.1],
       [0. , 1.7, 0.4],
       [0.3, 1. , 0. ],
       [2. , 2.3, 0.1],
       [0. , 0. , 0.3],
       [2.4, 5.8, 0.4],
       [0.4, 0. , 0. ],
       [1. , 1.7, 0.2],
       [1.1, 1.9, 0.3],
       [0.1, 1.4, 0.2],
       [1.1, 1.4, 0.1],
       [0.8, 1.2, 0.2],
       [0. , 3.6, 0.7],
       [1.6, 4.3, 0.2],
       [1.3, 3.7, 1.1],
       [2.3, 1.5, 0.2]])

In [8]:
test_label_df.head() # 정답 레이블은 2차원일 필요없으므로 ravel()로 1차원으로 펴주기!

,Pos
128,SG
15,C
132,C
29,SG
63,C


In [9]:
test_label_df.values # 2차원 배열 (values) ==> 1차원으로 펴주기

array([['SG'],
       ['C'],
       ['C'],
       ['SG'],
       ['C'],
       ['C'],
       ['SG'],
       ['SG'],
       ['C'],
       ['C'],
       ['C'],
       ['SG'],
       ['C'],
       ['C'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['C'],
       ['C'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['SG'],
       ['SG'],
       ['C'],
       ['SG'],
       ['C'],
       ['SG']], dtype=object)

In [10]:
test_label # 1차원 배열으로 펴진 것 (values & ravel())

array(['SG', 'C', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'C', 'C', 'SG',
       'C', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'SG', 'C', 'SG', 'SG', 'SG', 'C', 'SG', 'SG',
       'SG', 'C', 'SG', 'C', 'SG'], dtype=object)

### 5. KNN 모델 학습하기 (fit)
K-최근접 이웃 알고리즘으로 모델 학습시키기(**fit**)
* from sklearn.neighbors import KNeighborsClassifier
* **knn = KNeighborsClassifier(n_neighbors=num)**
* **knn.fit(train_data, train_label)**
> train_data(2차원 배열), train_label(1차원 배열)

In [11]:
knn = KNeighborsClassifier(n_neighbors=5) #knn 객체(분류기) 생성 (이웃 : 5개)
knn.fit(train_data, train_label)    #knn 모델로 학습하기 (fit)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

### 6. SVM 모델 학습하기 (fit)
서포트 벡터 머신 알고리즘으로 모델 학습시키기(**fit**)
* from sklearn import svm
* **clf = svm.SVC(C=cost, gamma=gamma)**
* **clf.fit(train_data, train_label)**

In [12]:
clf = svm.SVC(C = 1, gamma = 1) #svm 객체 생성
clf.fit(train_data, train_label)  #svm 모델로 학습하기

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### 7. 테스트 데이터로 예측하기 (predict)
학습이 완료된 모델에 test data를 넣어 예측해보기 (*1d array로 return*)
* **model.predict(test_data)**

In [13]:
pred_knn = knn.predict(test_data) #테스트데이터로 예측
pred_knn

array(['SG', 'C', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'C', 'C', 'SG',
       'C', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'SG', 'SG', 'SG', 'SG', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'SG', 'SG'], dtype=object)

In [14]:
pred_svm = clf.predict(test_data)
pred_svm

array(['SG', 'C', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'C', 'C', 'SG',
       'C', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'SG', 'SG', 'SG', 'SG', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'C', 'SG'], dtype=object)

### 8. 모델 예측 정확도 확인 (accuracy_score)
test set의 정답과 모델 예측값이 얼마나 일치하는지 정확도(accuracy) 계산
* **metrics.accuracy_score(test_label, pred_model)**

In [15]:
test_label #테스트 데이터 정답

array(['SG', 'C', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'C', 'C', 'SG',
       'C', 'C', 'SG', 'SG', 'C', 'SG', 'C', 'C', 'SG', 'SG', 'C', 'SG',
       'SG', 'C', 'SG', 'SG', 'C', 'SG', 'SG', 'SG', 'C', 'SG', 'SG',
       'SG', 'C', 'SG', 'C', 'SG'], dtype=object)

In [16]:
ac_score = metrics.accuracy_score(test_label, pred_knn)
print('accuracy : ', ac_score)
ac_score = metrics.accuracy_score(test_label, pred_svm)
print('accuracy : ', ac_score)

accuracy :  0.9
accuracy :  0.925


In [17]:
# DataFrame형식으로 만들어서 두 모델의 예측값들(1d array)과 정답 비교
comparison = pd.DataFrame({'prediction_knn':pred_knn, 'prediction_svm':pred_svm, 'ground_truth':test_label})
comparison

,prediction_knn,prediction_svm,ground_truth
0,SG,SG,SG
1,C,C,C
2,C,C,C
3,SG,SG,SG
4,C,C,C
5,C,C,C
6,SG,SG,SG
7,SG,SG,SG
8,C,C,C
9,C,C,C
